<a href="https://colab.research.google.com/github/PeiYauWeng/SpeechNet_based_on_Google_WaveNet/blob/master/speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hickle

In [ ]:
%pdb

Automatic pdb calling has been turned ON


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ln -s "/content/drive/My Drive/Colab Notebooks/asset"/data

Mounted at /content/drive
ln: failed to create symbolic link './data': File exists


In [ ]:
!ls data

0d2bcf9d_nohash_1.wav		6b7d5101_nohash_0.wav
0e17f595_nohash_0.wav		6b81fead_nohash_0.wav
0e17f595_nohash_1.wav		6b81fead_nohash_1.wav
11b1df78_nohash_0.wav		6b81fead_nohash_2.wav
11b1df78_nohash_1.wav		6d1dcca6_nohash_0.wav
11b1df78_nohash_1.wav.wave.hkl	6d1dcca6_nohash_1.wav
171b56dc_nohash_0.wav		6d818f6c_nohash_1.wav
17f0e206_nohash_0.wav		6d818f6c_nohash_2.wav
17f0e206_nohash_0.wav.wave.hkl	6f5eea74_nohash_0.wav
17f0e206_nohash_1.wav		708a9569_nohash_0.wav
1a4259c3_nohash_1.wav		708a9569_nohash_1.wav
1b63157b_nohash_0.wav		742d6431_nohash_0.wav
1b755c65_nohash_0.wav		742d6431_nohash_0.wav.wave.hkl
1b755c65_nohash_1.wav		789e4ee7_nohash_0.wav
1df483c0_nohash_0.wav		7ff085e1_nohash_0.wav
229978fd_nohash_0.wav		7ff085e1_nohash_1.wav
229978fd_nohash_0.wav.wave.hkl	8012c69d_nohash_0.wav
26e573a9_nohash_0.wav		810c99be_nohash_0.wav
26e573a9_nohash_0.wav.wave.hkl	8134f43f_nohash_0.wav
28497c5b_nohash_0.wav		84bf12ff_nohash_0.wav
2a0b413e_nohash_0.wav		84d1e469_nohash_0.wav
2aec99ec_nohas

In [ ]:
%matplotlib inline
%tensorflow_version 1.x

In [ ]:
import os

os.chdir('..')

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
EAGER=False
RUN_TESTS=False
PRINT_ALPHABET=False

In [ ]:
if EAGER:
    tf.enable_eager_execution()

In [ ]:
SAMPLING_RATE=16000

In [ ]:
def to_path(filename):
    return './data/' + filename

In [ ]:
def random_stretch(audio, params):
    """
    Stretches randomly the input audio
    """
    
    rate = random.uniform(params['random_stretch_min'], params['random_stretch_max'])
    
    return librosa.effects.time_stretch(audio, rate)

In [ ]:
import random

def random_shift(audio, params):
    """
    Shifts randomly the input audio to the left or the right
    """
    
    _shift = random.randrange(params['random_shift_min'], params['random_shift_max'])
    
    if _shift < 0:
        pad = (_shift * -1, 0)
    else:
        pad = (0, _shift)
    
    return np.pad(audio, pad, mode='constant')

In [ ]:
import glob

noise_files = glob.glob('./data/*.wav')
noises = {}

def random_noise(audio, params):
    _factor = random.uniform(
        params['random_noise_factor_min'],
        params['random_noise_factor_max']
    )    
    
    if params['random_noise'] > random.uniform(0, 1):
        _path = random.choice(noise_files)
        
        if _path in noises:
            wave = noises[_path]
        else:
            if os.path.isfile(_path + '.wave.hkl'):
                wave = hkl.load(_path + '.wave.hkl').astype(np.float32)
                noises[_path] = wave
            else:
                wave, _ = librosa.load(_path, sr=SAMPLING_RATE)
                hkl.dump(wave, _path + '.wave.hkl')
                noises[_path] = wave

        noise = random_shift(
            wave,
            {
                'random_shift_min': -16000,
                'random_shift_max': 16000
            }
        )
        
        max_noise = np.max(noise[0:len(audio)])
        max_wave = np.max(audio)
        
        noise = noise * (max_wave / max_noise)
        #print(_factor);
        #print(noise[0:len(audio)]);
        #print(audio);
        if len(noise)>= len(audio):
          return _factor * noise[0:len(audio)] + (1.0 - _factor) * audio
        else:
          return _factor * noise + (1.0 - _factor) * audio[0:len(noise)]
    else:
        return audio

In [ ]:
import librosa
import hickle as hkl
import os.path

def load_wave(example, absolute=False):
    row, params = example
    
    _path = row.filename if absolute else to_path(row.filename)
    
    if os.path.isfile(_path + '.wave.hkl'):
        wave = hkl.load(_path + '.wave.hkl').astype(np.float32)
    else:
        wave, _ = librosa.load(_path, sr=SAMPLING_RATE)
        hkl.dump(wave, _path + '.wave.hkl')

    if len(wave) <= params['max_wave_length']:
        if params['augment'] and row.filename.split('/')[0] != 'voxforge':
            wave = random_noise(
                random_stretch(
                    random_shift(
                        wave,
                        params
                    ),
                    params
                ),
                params
            )
    else:
        wave = None
    
    return wave, row

In [ ]:
#from IPython.display import Audio
#
#params = {
#    'augment': True,
#    'random_shift_min': -4000,
#    'random_shift_max': 4000,
#    'random_noise': 1.0,
#    'random_noise_factor_min': 0.1,
#    'random_noise_factor_max': 0.15,
#    'random_stretch_min': 0.8,
#    'random_stretch_max': 1.2
#}
#
#_, row = next(train_data.iterrows())
#
#wave, _ = load_wave((row, params))
#
#Audio(data=wave, rate=16000)

In [ ]:
#import os.path
#import glob
#
#with_length_filename = 'with_lengths.csv'
#
#if not os.path.isfile(with_length_filename):
#    train_eval_data = pd.read_csv('./data/cv_corpus_v1/cv-valid-train.csv')
#    
#    def get_len(row):
#        wave, _ = load_wave(row)
#
#        return len(wave)
#
#    lengths = [
#        get_len(row)
#        for _, row in train_eval_data.iterrows()
#    ]
#
#    hkl.dump(lengths, 'lengths.hkl')
#    
#    train_eval_data['length'] = lengths
#    
#    train_eval_data.to_csv('with_lengths.csv')
#else:
#    train_eval_data = pd.read_csv(with_length_filename)

In [ ]:
train_eval_data = pd.read_csv('./data/cv_corpus_v1/train.tsv', sep='\t',header=0)
train_eval_data

,client_id,path,sentence,up_votes,down_votes,age,gender,accent
0,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664034.mp3,"These data components in turn serve as the ""bu...",2,0,thirties,male,NaN
1,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664035.mp3,The church is unrelated to the Jewish politica...,3,0,thirties,male,NaN
2,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664037.mp3,The following represents architectures which h...,2,0,thirties,male,NaN
3,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664038.mp3,"Additionally, the pulse output can be directed...",2,0,thirties,male,NaN
4,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664040.mp3,The two are robbed by a pickpocket who is losi...,3,0,thirties,male,NaN
...,...,...,...,...,...,...,...,...
232970,939991dfcf12d8250739f3b4e5ed47e0dc4bf8ffb69810...,common_voice_en_19753579.mp3,He has two older half-brothers from his mother...,2,0,twenties,male,NaN
232971,939991dfcf12d8250739f3b4e5ed47e0dc4bf8ffb69810...,common_voice_en_19753580.mp3,Alessandria is also a major railway hub.,2,0,twenties,male,NaN
232972,939991dfcf12d8250739f3b4e5ed47e0dc4bf8ffb69810...,common_voice_en_19753581.mp3,This remains a record number of goals scored i...,2,0,twenties,male,NaN
232973,939991dfcf12d8250739f3b4e5ed47e0dc4bf8ffb69810...,common_voice_en_19753589.mp3,"As a result, he was offered a recording contra...",2,1,twenties,male,NaN


In [ ]:
train_eval_data = train_eval_data[['path','sentence']]
train_eval_data = train_eval_data.rename(columns={'path':'filename','sentence':'text'})
train_eval_data

,filename,text
0,common_voice_en_19664034.mp3,"These data components in turn serve as the ""bu..."
1,common_voice_en_19664035.mp3,The church is unrelated to the Jewish politica...
2,common_voice_en_19664037.mp3,The following represents architectures which h...
3,common_voice_en_19664038.mp3,"Additionally, the pulse output can be directed..."
4,common_voice_en_19664040.mp3,The two are robbed by a pickpocket who is losi...
...,...,...
232970,common_voice_en_19753579.mp3,He has two older half-brothers from his mother...
232971,common_voice_en_19753580.mp3,Alessandria is also a major railway hub.
232972,common_voice_en_19753581.mp3,This remains a record number of goals scored i...
232973,common_voice_en_19753589.mp3,"As a result, he was offered a recording contra..."


In [ ]:
if not os.path.isfile('./data/train.csv'):
    eval_data = train_eval_data.sample(n=int(len(train_eval_data) * 0.1 ))
    train_data = train_eval_data[~train_eval_data.isin(eval_data)]
    train_data = train_data[train_data.filename.notnull()]
    
    train_data.to_csv('./data/train.csv')
    eval_data.to_csv('./data/eval.csv')
else:
    train_data = pd.read_csv('./data/train.csv')
    eval_data = pd.read_csv('./data/eval.csv')

In [ ]:
!ls data/

0d2bcf9d_nohash_1.wav  590750e8_nohash_1.wav  86402da1_nohash_0.wav
0e17f595_nohash_0.wav  591d32f3_nohash_0.wav  881583a6_nohash_0.wav
0e17f595_nohash_1.wav  591d32f3_nohash_1.wav  8a28231e_nohash_0.wav
11b1df78_nohash_0.wav  5b1db3ee_nohash_0.wav  8b775397_nohash_0.wav
11b1df78_nohash_1.wav  5b1db3ee_nohash_2.wav  90b0b91a_nohash_0.wav
171b56dc_nohash_0.wav  5b1db3ee_nohash_3.wav  92b2bf59_nohash_0.wav
17f0e206_nohash_0.wav  5c8af87a_nohash_0.wav  92b2bf59_nohash_2.wav
17f0e206_nohash_1.wav  5e033479_nohash_0.wav  93ec8b84_nohash_0.wav
1a4259c3_nohash_1.wav  5e033479_nohash_1.wav  93ec8b84_nohash_1.wav
1b63157b_nohash_0.wav  620ff0fa_nohash_0.wav  98582fee_nohash_0.wav
1b755c65_nohash_0.wav  62ff07ef_nohash_0.wav  98582fee_nohash_1.wav
1b755c65_nohash_1.wav  65c73b55_nohash_0.wav  988e2f9a_nohash_0.wav
1df483c0_nohash_0.wav  65d844fc_nohash_0.wav  988e2f9a_nohash_1.wav
229978fd_nohash_0.wav  65ec06e5_nohash_0.wav  9a4d12fd_nohash_0.wav
26e573a9_nohash_0.wav  65f2531f_nohash_0.wav  9a

In [ ]:
#import seaborn as sb
#
#lens = []
#libri_count = 0
#
#for _, row in libri.iterrows():
#    wave, _ = load_wave((row, { 'augment': False, 'max_wave_length': 320000 }))
#    if wave is not None:
#        libri_count += 1
#        lens.append(len(wave))
#        
#print(libri_count)
#sb.distplot(lens, kde = False)

In [ ]:
#import seaborn as sb
#
#lens = []
#vox_count = 0
#
#for _, row in vox.iterrows():
#    wave, _ = load_wave((row, { 'augment': False, 'max_wave_length': 320000 }))
#    if wave is not None:
#        vox_count += 1
#        lens.append(len(wave))
#        
#print(vox_count)
#sb.distplot(lens, kde = False)

In [ ]:
if not os.path.isfile('./data/libri.csv'):
    def libri_texts():
        for path in glob.glob('./data/LibriSpeech/**/**/**/*.txt'):
            file_prefix = '/'.join(path.split('/')[0:-1])

            with open(path, 'r') as file:
                lines = file.readlines()

                for line in lines:
                    elements = line.split(' ')
                    prefix = elements[0]
                    text = ' '.join(elements[1:]).replace('\n', '').lower()

                    yield '{}/{}.flac'.format(file_prefix.replace('./data/', ''), prefix), text

    libri_frame = pd.DataFrame([ (a, b) for a, b in libri_texts() ], columns=['filename', 'text'])
    libri_frame.to_csv('./data/libri.csv')
else:
    libri_frame = pd.read_csv('./data/libri.csv')

libri_frame

,Unnamed: 0,filename,text
0,0,LibriSpeech/dev-clean/1272/128104/1272-128104-...,mister quilter is the apostle of the middle cl...
1,1,LibriSpeech/dev-clean/1272/128104/1272-128104-...,nor is mister quilter's manner less interestin...
2,2,LibriSpeech/dev-clean/1272/128104/1272-128104-...,he tells us that at this festive season of the...
3,3,LibriSpeech/dev-clean/1272/128104/1272-128104-...,he has grave doubts whether sir frederick leig...
4,4,LibriSpeech/dev-clean/1272/128104/1272-128104-...,linnell's pictures are a sort of up guards and...
...,...,...,...
96369,96369,LibriSpeech/train-clean-360/8138/293529/8138-2...,sure he will said mc givney that's the point y...
96370,96370,LibriSpeech/train-clean-360/8138/293529/8138-2...,the address of young lackman was the hotel de ...
96371,96371,LibriSpeech/train-clean-360/8138/293529/8138-2...,all life was a gigantic bluff and you encourag...
96372,96372,LibriSpeech/train-clean-360/8138/293529/8138-2...,the blue uniformed guardians drew them back wi...


In [ ]:
!ls data/voxforge_corpus/k-20090202-ncv/etc

audiofile_details  HDMan_log  PROMPTS		README
GPL_license.txt    HVite_log  prompts-original


In [ ]:
if not os.path.isfile('./data/vox.csv'):
    def vox_texts():
        for path in glob.glob('./data/voxforge_corpus/**/**/PROMPTS'):
            file_prefix = '/'.join(path.split('/')[0:-2])

            with open(path, 'r') as file:
                lines = file.readlines()

                if 'wav' in [ s.split('/')[-1] for s in glob.glob('{}/*'.format(file_prefix))]:
                    form = 'wav'
                else:
                    form = 'flac'

                for line in lines:
                    elements = line.split(' ')
                    prefix = elements[0].split('/')[-1]
                    text = ' '.join(elements[1:]).replace('\n', '').lower()
                    
                    fn = '{}/{}/{}.{}'.format(file_prefix.replace('./data/', ''), form, prefix, form)
                    
                    #import pdb; pdb.set_trace()
                    
                    if os.path.isfile('./data/{}'.format(fn)):
                        yield fn, text

    vox_frame = pd.DataFrame([ (a, b) for a, b in vox_texts() ], columns=['filename', 'text'])
    vox_frame.to_csv('./data/vox.csv')
else:
    vox_frame = pd.read_csv('./data/vox.csv')

vox_frame

,Unnamed: 0,filename,text
0,0,voxforge_corpus/1028-20100710-hne/wav/rp-29.wav,since red and green light when mixed form yellow
1,1,voxforge_corpus/1028-20100710-hne/wav/rp-30.wav,this is a very common type of bow one showing ...
2,2,voxforge_corpus/1028-20100710-hne/wav/rp-31.wav,with little or no green or blue
3,3,voxforge_corpus/1028-20100710-hne/wav/ar-01.wav,once there was a young rat named arthur who ne...
4,4,voxforge_corpus/1028-20100710-hne/wav/ar-02.wav,whenever his friends asked him if he would lik...
...,...,...,...
62909,62909,voxforge_corpus/glenn-20080119-kvg/wav/b0003.wav,i can see that knife now
62910,62910,voxforge_corpus/glenn-20080119-kvg/wav/b0004.wav,when i can't see beauty in woman i want to die
62911,62911,voxforge_corpus/glenn-20080119-kvg/wav/b0005.wav,his slim fingers closed like steel about philip's
62912,62912,voxforge_corpus/glenn-20080119-kvg/wav/b0006.wav,he seized gregson by the arm and led him to th...


In [ ]:
!ls data

cv_corpus_v1	      exercise_bike.wav  running_tap.wav   vox.csv
doing_the_dishes.wav  libri.csv		 TEDLIUM_release2  voxforge_corpus
dude_miaowing.wav     LibriSpeech	 train.csv	   white_noise.wav
eval.csv	      pink_noise.wav	 VCTK-Corpus


In [ ]:
test_data = pd.read_csv('./data/cv_corpus_v1/test.tsv', sep='\t', header=0)
test_data

,client_id,path,sentence,up_votes,down_votes,age,gender,accent
0,0013037a1d45cc33460806cc3f8ecee9d536c45639ba4c...,common_voice_en_699711.mp3,She'll be all right.,2,1,NaN,NaN,NaN
1,001509f4624a7dee75247f6a8b642c4a0d09f8be3eeea6...,common_voice_en_18132047.mp3,All's well that ends well.,2,0,NaN,NaN,NaN
2,003fb666a99eb3aa3ba05d9c8641c18e55cf7d34d1b981...,common_voice_en_17263741.mp3,Do you mean it?,2,0,NaN,NaN,NaN
3,004017ba82a23768d58dff3b91da8e8f951ea5fb6d3cd9...,common_voice_en_17893917.mp3,The new patch is less invasive than the old on...,2,1,NaN,NaN,NaN
4,0047f1aea3f39c4c6a9298d84f046c1f84f439f594d840...,common_voice_en_17561821.mp3,How is Mozilla going to handle ambiguities lik...,2,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
15526,d24bb5a2c0b11abbc6ad33575db579f576f26fdbe7aad5...,common_voice_en_19631259.mp3,Both lawsuits were subsequently dropped by the...,3,0,NaN,NaN,NaN
15527,d24bb5a2c0b11abbc6ad33575db579f576f26fdbe7aad5...,common_voice_en_19631262.mp3,"But what of Spiller, the Man of Action?",2,0,NaN,NaN,NaN
15528,d24bb5a2c0b11abbc6ad33575db579f576f26fdbe7aad5...,common_voice_en_19631263.mp3,The town is also the site of a sausage festival.,2,0,NaN,NaN,NaN
15529,d275f2d0f87acb8af42564d63fa6120d5cc54eb3f249f7...,common_voice_en_17361996.mp3,Did the dog sniff anyone out yet?,2,0,NaN,NaN,NaN


In [ ]:
test_data = test_data[['path','sentence']]
test_data = test_data.rename(columns={'path':'filename','sentence':'text'})
test_data

,filename,text
0,common_voice_en_699711.mp3,She'll be all right.
1,common_voice_en_18132047.mp3,All's well that ends well.
2,common_voice_en_17263741.mp3,Do you mean it?
3,common_voice_en_17893917.mp3,The new patch is less invasive than the old on...
4,common_voice_en_17561821.mp3,How is Mozilla going to handle ambiguities lik...
...,...,...
15526,common_voice_en_19631259.mp3,Both lawsuits were subsequently dropped by the...
15527,common_voice_en_19631262.mp3,"But what of Spiller, the Man of Action?"
15528,common_voice_en_19631263.mp3,The town is also the site of a sausage festival.
15529,common_voice_en_17361996.mp3,Did the dog sniff anyone out yet?


In [ ]:
train_data['filename'] = train_data['filename'].apply(lambda f: 'cv_corpus_v1/clips/{}'.format(f))
train_data

,Unnamed: 0,filename,text
0,0,cv_corpus_v1/clips/common_voice_en_19664034.mp3,"These data components in turn serve as the ""bu..."
1,1,cv_corpus_v1/clips/common_voice_en_19664035.mp3,The church is unrelated to the Jewish politica...
2,2,cv_corpus_v1/clips/common_voice_en_19664037.mp3,The following represents architectures which h...
3,3,cv_corpus_v1/clips/common_voice_en_19664038.mp3,"Additionally, the pulse output can be directed..."
4,4,cv_corpus_v1/clips/common_voice_en_19664040.mp3,The two are robbed by a pickpocket who is losi...
...,...,...,...
209673,232970,cv_corpus_v1/clips/common_voice_en_19753579.mp3,He has two older half-brothers from his mother...
209674,232971,cv_corpus_v1/clips/common_voice_en_19753580.mp3,Alessandria is also a major railway hub.
209675,232972,cv_corpus_v1/clips/common_voice_en_19753581.mp3,This remains a record number of goals scored i...
209676,232973,cv_corpus_v1/clips/common_voice_en_19753589.mp3,"As a result, he was offered a recording contra..."


In [ ]:
eval_data['filename'] = eval_data['filename'].apply(lambda f: 'cv_corpus_v1/clips/{}'.format(f))
test_data['filename'] = test_data['filename'].apply(lambda f: 'cv_corpus_v1/clips/{}'.format(f))

In [ ]:
test_data

,filename,text
0,cv_corpus_v1/clips/common_voice_en_699711.mp3,She'll be all right.
1,cv_corpus_v1/clips/common_voice_en_18132047.mp3,All's well that ends well.
2,cv_corpus_v1/clips/common_voice_en_17263741.mp3,Do you mean it?
3,cv_corpus_v1/clips/common_voice_en_17893917.mp3,The new patch is less invasive than the old on...
4,cv_corpus_v1/clips/common_voice_en_17561821.mp3,How is Mozilla going to handle ambiguities lik...
...,...,...
15526,cv_corpus_v1/clips/common_voice_en_19631259.mp3,Both lawsuits were subsequently dropped by the...
15527,cv_corpus_v1/clips/common_voice_en_19631262.mp3,"But what of Spiller, the Man of Action?"
15528,cv_corpus_v1/clips/common_voice_en_19631263.mp3,The town is also the site of a sausage festival.
15529,cv_corpus_v1/clips/common_voice_en_17361996.mp3,Did the dog sniff anyone out yet?


In [ ]:
if not os.path.isfile('./data/full_train.csv'):
    ##train_data = train_data[['filename', 'text']].append(libri_frame).append(vox_frame)
    train_data = libri_frame.append(vox_frame)
    train_data.to_csv('./data/full_train.csv')
else:
    train_data = pd.read_csv('./data/full_train.csv')

train_data = train_data[['filename','text']]
train_data


,filename,text
0,LibriSpeech/dev-clean/1272/128104/1272-128104-...,mister quilter is the apostle of the middle cl...
1,LibriSpeech/dev-clean/1272/128104/1272-128104-...,nor is mister quilter's manner less interestin...
2,LibriSpeech/dev-clean/1272/128104/1272-128104-...,he tells us that at this festive season of the...
3,LibriSpeech/dev-clean/1272/128104/1272-128104-...,he has grave doubts whether sir frederick leig...
4,LibriSpeech/dev-clean/1272/128104/1272-128104-...,linnell's pictures are a sort of up guards and...
...,...,...
159283,voxforge_corpus/glenn-20080119-kvg/wav/b0003.wav,i can see that knife now
159284,voxforge_corpus/glenn-20080119-kvg/wav/b0004.wav,when i can't see beauty in woman i want to die
159285,voxforge_corpus/glenn-20080119-kvg/wav/b0005.wav,his slim fingers closed like steel about philip's
159286,voxforge_corpus/glenn-20080119-kvg/wav/b0006.wav,he seized gregson by the arm and led him to th...


In [ ]:
!ls data

0d2bcf9d_nohash_1.wav  590750e8_nohash_1.wav  86402da1_nohash_0.wav
0e17f595_nohash_0.wav  591d32f3_nohash_0.wav  881583a6_nohash_0.wav
0e17f595_nohash_1.wav  591d32f3_nohash_1.wav  8a28231e_nohash_0.wav
11b1df78_nohash_0.wav  5b1db3ee_nohash_0.wav  8b775397_nohash_0.wav
11b1df78_nohash_1.wav  5b1db3ee_nohash_2.wav  90b0b91a_nohash_0.wav
171b56dc_nohash_0.wav  5b1db3ee_nohash_3.wav  92b2bf59_nohash_0.wav
17f0e206_nohash_0.wav  5c8af87a_nohash_0.wav  92b2bf59_nohash_2.wav
17f0e206_nohash_1.wav  5e033479_nohash_0.wav  93ec8b84_nohash_0.wav
1a4259c3_nohash_1.wav  5e033479_nohash_1.wav  93ec8b84_nohash_1.wav
1b63157b_nohash_0.wav  620ff0fa_nohash_0.wav  98582fee_nohash_0.wav
1b755c65_nohash_0.wav  62ff07ef_nohash_0.wav  98582fee_nohash_1.wav
1b755c65_nohash_1.wav  65c73b55_nohash_0.wav  988e2f9a_nohash_0.wav
1df483c0_nohash_0.wav  65d844fc_nohash_0.wav  988e2f9a_nohash_1.wav
229978fd_nohash_0.wav  65ec06e5_nohash_0.wav  9a4d12fd_nohash_0.wav
26e573a9_nohash_0.wav  65f2531f_nohash_0.wav  9a

In [ ]:
# we need to gather the alphabet to use:
def print_alphabet():
    uniques = set()
    
    data_frames = [
        train_data,
        test_data
    ]
    print(data_frames)
    for ix, data_frame in enumerate(data_frames):
        #print("Gathering unique characters for set no. " + str(ix + 1) + " (of " + str(len(data_frames)) + " all)") 

        train_data.apply(lambda row: [ c for c in row.text ], axis=1).apply(uniques.update)
        
    print(sorted(uniques))
    print(''.join(sorted(uniques)))

if PRINT_ALPHABET:
    print_alphabet()

In [ ]:
print_alphabet()

[                                                 filename                                               text
0       LibriSpeech/dev-clean/1272/128104/1272-128104-...  mister quilter is the apostle of the middle cl...
1       LibriSpeech/dev-clean/1272/128104/1272-128104-...  nor is mister quilter's manner less interestin...
2       LibriSpeech/dev-clean/1272/128104/1272-128104-...  he tells us that at this festive season of the...
3       LibriSpeech/dev-clean/1272/128104/1272-128104-...  he has grave doubts whether sir frederick leig...
4       LibriSpeech/dev-clean/1272/128104/1272-128104-...  linnell's pictures are a sort of up guards and...
...                                                   ...                                                ...
159283   voxforge_corpus/glenn-20080119-kvg/wav/b0003.wav                           i can see that knife now
159284   voxforge_corpus/glenn-20080119-kvg/wav/b0004.wav     when i can't see beauty in woman i want to die
159285   voxforge_

In [ ]:
def compute_lengths(original_lengths, params):
    """
    Computes the length of data for CTC
    """
    
    return tf.cast(
        tf.floor(
            (tf.cast(original_lengths, dtype=tf.float32) - params['n_fft']) /
                params['frame_step']
        ) + 1,
        tf.int32
    )

In [ ]:
def encode_labels(labels, params):
    characters = list(params['alphabet'])
    
    table = tf.contrib.lookup.HashTable(
        tf.contrib.lookup.KeyValueTensorInitializer(
            characters,
            list(range(len(characters)))
        ),
        -1,
        name='char2id'
    )
    
    return table.lookup(
        tf.string_split(labels, delimiter='')
    )

In [ ]:
def decode_codes(codes, params):
    characters = list(params['alphabet'])
    
    table = tf.contrib.lookup.HashTable(
        tf.contrib.lookup.KeyValueTensorInitializer(
            list(range(len(characters))),
            characters
        ),
        '',
        name='id2char'
    )
    
    return table.lookup(codes)

In [ ]:
def decode_logits(logits, lengths, params):
    if len(tf.shape(lengths).shape) == 1:
        lengths = tf.reshape(lengths, [1])
    else:
        lengths = tf.squeeze(lengths)
        
    predicted_codes, _ = tf.nn.ctc_beam_search_decoder(
        tf.transpose(logits, (1, 0, 2)),
        lengths,
        merge_repeated=True
    )
    
    codes = tf.cast(predicted_codes[0], tf.int32)
    
    text = decode_codes(codes, params)
    
    return text, codes

In [ ]:
class LogMelSpectrogram(tf.layers.Layer):
    def __init__(self,
                 sampling_rate,
                 n_fft,
                 frame_step,
                 lower_edge_hertz,
                 upper_edge_hertz,
                 num_mel_bins,
                 **kwargs):
        super(LogMelSpectrogram, self).__init__(**kwargs)
        
        self.sampling_rate = sampling_rate
        self.n_fft = n_fft
        self.frame_step = frame_step
        self.lower_edge_hertz = lower_edge_hertz
        self.upper_edge_hertz = upper_edge_hertz
        self.num_mel_bins = num_mel_bins
        
    def call(self, inputs, training=True):
        stfts = tf.contrib.signal.stft(
            inputs,
            frame_length=self.n_fft,
            frame_step=self.frame_step,
            fft_length=self.n_fft,
            pad_end=False
        )
        
        power_spectrograms = tf.real(stfts * tf.conj(stfts))
        
        num_spectrogram_bins = power_spectrograms.shape[-1].value
    
        linear_to_mel_weight_matrix = tf.constant(
            np.transpose(
                librosa.filters.mel(
                    sr=self.sampling_rate,
                    n_fft=self.n_fft + 1,
                    n_mels=self.num_mel_bins,
                    fmin=self.lower_edge_hertz,
                    fmax=self.upper_edge_hertz
                )
            ),
            dtype=tf.float32
        )
        
        mel_spectrograms = tf.tensordot(
            power_spectrograms,
            linear_to_mel_weight_matrix,
            1
        )
        
        mel_spectrograms.set_shape(
            power_spectrograms.shape[:-1].concatenate(
                linear_to_mel_weight_matrix.shape[-1:]
            )
        )
        
        return tf.log(mel_spectrograms + 1e-6)

In [ ]:
class AtrousConv1D(tf.layers.Layer):
    def __init__(self,
                 filters,
                 kernel_size,
                 dilation_rate,
                 use_bias=True,
                 kernel_initializer=tf.glorot_normal_initializer(),
                 causal=True
                ):
        super(AtrousConv1D, self).__init__()
        
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.causal = causal
        
        self.conv1d = tf.layers.Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            padding='valid' if causal else 'same',
            use_bias=use_bias,
            kernel_initializer=kernel_initializer
        )
        
    def call(self, inputs):
        if self.causal:
            padding = (self.kernel_size - 1) * self.dilation_rate
            inputs = tf.pad(inputs, tf.constant([(0, 0,), (1, 0), (0, 0)]) * padding)
        
        return self.conv1d(inputs)

In [ ]:
class ResidualBlock(tf.layers.Layer):
    def __init__(self, filters, kernel_size, dilation_rate, causal, **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        
        self.dilated_conv1 = AtrousConv1D(
            filters=filters,
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            causal=causal
        )
        
        self.dilated_conv2 = AtrousConv1D(
            filters=filters,
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            causal=causal
        )
        
        self.out = tf.layers.Conv1D(
            filters=filters,
            kernel_size=1
        )
        
    def call(self, inputs, training=True):
        data = tf.layers.batch_normalization(
            inputs,
            training=training
        )
        
        filters = self.dilated_conv1(data)
        gates = self.dilated_conv2(data)
        
        filters = tf.nn.tanh(filters)
        gates = tf.nn.sigmoid(gates)
        
        out = tf.nn.tanh(
            self.out(
                filters * gates
            )
        )
        
        return out + inputs, out

In [ ]:
class ResidualStack(tf.layers.Layer):
    def __init__(self, filters, kernel_size, dilation_rates, causal, **kwargs):
        super(ResidualStack, self).__init__(**kwargs)
        
        self.blocks = [
            ResidualBlock(
                filters=filters,
                kernel_size=kernel_size,
                dilation_rate=dilation_rate,
                causal=causal
            )
            for dilation_rate in dilation_rates
        ]
        
    def call(self, inputs, training=True):
        data = inputs
        skip = 0
        
        for block in self.blocks:
            data, current_skip = block(data, training=training)
            skip += current_skip

        return skip

In [ ]:
class SpeechNet(tf.layers.Layer):
    def __init__(self, params, **kwargs):
        super(SpeechNet, self).__init__(**kwargs)
        
        self.to_log_mel = LogMelSpectrogram(
            sampling_rate=params['sampling_rate'],
            n_fft=params['n_fft'],
            frame_step=params['frame_step'],
            lower_edge_hertz=params['lower_edge_hertz'],
            upper_edge_hertz=params['upper_edge_hertz'],
            num_mel_bins=params['num_mel_bins']
        )
        
        self.expand = tf.layers.Conv1D(
            filters=params['stack_filters'],
            kernel_size=1,
            padding='same'
        )
        
        self.stacks = [
            ResidualStack(
                filters=params['stack_filters'],
                kernel_size=params['stack_kernel_size'],
                dilation_rates=params['stack_dilation_rates'],
                causal=params['causal_convolutions']
            )
            for _ in range(params['stacks'])
        ]
        
        self.out = tf.layers.Conv1D(
            filters=len(params['alphabet']) + 1,
            kernel_size=1,
            padding='same'
        )
        
    def call(self, inputs, training=True):
        data = self.to_log_mel(inputs)
        
        data = tf.layers.batch_normalization(
            data,
            training=training
        )
        
        if len(data.shape) == 2:
            data = tf.expand_dims(data, 0)
        
        data = self.expand(data)
        
        for stack in self.stacks:
            data = stack(data, training=training)
        
        data = tf.layers.batch_normalization(
            data,
            training=training
        )
        
        return self.out(data) + 1e-8

In [ ]:
from multiprocessing import Pool

def input_fn(input_dataset, params, load_wave_fn=load_wave):
    def _input_fn():
        """
        Returns raw audio wave along with the label
        """
        
        dataset = input_dataset
        
        print(params)
        
        if 'max_text_length' in params and params['max_text_length'] is not None:
            print('Constraining dataset to the max_text_length')
            dataset = input_dataset[input_dataset.text.str.len() < params['max_text_length']]
            
        if 'min_text_length' in params and params['min_text_length'] is not None:
            print('Constraining dataset to the min_text_length')
            dataset = input_dataset[input_dataset.text.str.len() >= params['min_text_length']]
            
        if 'max_wave_length' in params and params['max_wave_length'] is not None:
            print('Constraining dataset to the max_wave_length')
            
        print('Resulting dataset length: {}'.format(len(dataset)))
        
        def generator_fn():
            pool = Pool()
            buffer = []
            
            for epoch in range(params['epochs']):
                
                if params['shuffle']:
                    dataset = input_dataset
                    dataset = dataset.sample(frac=1)
                else:
                    dataset = input_dataset
                    
                for _, row in dataset.iterrows():
                    buffer.append((row, params))

                    if len(buffer) >= params['batch_size']:

                        if params['parallelize']:
                            audios = pool.map(
                                load_wave_fn,
                                buffer
                            )
                        else:
                            audios = map(
                                load_wave_fn,
                                buffer
                            )

                        for audio, row in audios:
                            if audio is not None:
                                if np.isnan(audio).any():
                                    print('SKIPPING! NaN coming from the pipeline!')
                                else:
                                    #print(row.text)
                                    yield (audio, len(audio)), row.text.encode()

                        buffer = []

        return tf.data.Dataset.from_generator(
                generator_fn,
                output_types=((tf.float32, tf.int32), (tf.string)),
                output_shapes=((None,()), (()))
            ) \
            .padded_batch(
                batch_size=params['batch_size'],
                padded_shapes=(
                    (tf.TensorShape([None]), tf.TensorShape(())),
                    tf.TensorShape(())
                )
            )
    
    return _input_fn

In [ ]:
def model_fn(features, labels, mode, params):
    if isinstance(features, dict):
        audio = features['audio']
        original_lengths = features['length']
    else:
        audio, original_lengths = features

    lengths = compute_lengths(original_lengths, params)
    
    if labels is not None:
        codes = encode_labels(labels, params)

    network = SpeechNet(params)

    is_training = mode==tf.estimator.ModeKeys.TRAIN
    
    print('Is training? {}'.format(is_training))

    logits = network(audio, training=is_training)
    text, predicted_codes = decode_logits(logits, lengths, params)

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'logits': logits,
            'text': tf.sparse_tensor_to_dense(
                text,
                ''
            )
        }

        export_outputs = {
            'predictions': tf.estimator.export.PredictOutput(predictions)
        }

        return tf.estimator.EstimatorSpec(
            mode,
            predictions=predictions,
            export_outputs=export_outputs
        )
    else:        
        loss = tf.reduce_mean(
            tf.nn.ctc_loss(
                labels=codes,
                inputs=logits,
                sequence_length=lengths,
                time_major=False,
                ignore_longer_outputs_than_inputs=True
            )
        )

        mean_edit_distance = tf.reduce_mean(
            tf.edit_distance(
                tf.cast(predicted_codes, tf.int32),
                codes
            )
        )

        distance_metric = tf.metrics.mean(mean_edit_distance)

        if mode == tf.estimator.ModeKeys.EVAL:            
            return tf.estimator.EstimatorSpec(
                mode,
                loss=loss,
                eval_metric_ops={ 'edit_distance': distance_metric }
            )

        elif mode == tf.estimator.ModeKeys.TRAIN:
            global_step = tf.train.get_or_create_global_step()

            tf.summary.text(
                'train_predicted_text',
                tf.sparse_tensor_to_dense(text, '')
            )
            tf.summary.scalar('train_edit_distance', mean_edit_distance)

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                train_op = tf.contrib.layers.optimize_loss(
                    loss=loss,
                    global_step=global_step,
                    learning_rate=params['lr'],
                    optimizer=(params['optimizer']),
                    update_ops=update_ops,
                    clip_gradients=params['clip_gradients'],
                    summaries=[
                        "learning_rate",
                        "loss",
                        "global_gradient_norm",
                    ]
                )

            return tf.estimator.EstimatorSpec(
                mode,
                loss=loss,
                train_op=train_op
            )

In [ ]:
def experiment_name(params, excluded_keys=['alphabet', 'data', 'lr', 'clip_gradients']):

    def represent(key, value):
        if key in excluded_keys:
            return None
        else:
            if isinstance(value, list):
                return '{}_{}'.format(key, '_'.join([str(v) for v in value]))
            else:
                return '{}_{}'.format(key, value)

    parts = filter(
        lambda p: p is not None,
        [
            represent(k, params[k])
            for k in sorted(params.keys())
        ]
    )

    return '/'.join(parts)

In [ ]:
def dataset_params(batch_size=32,
                   epochs=50000,
                   parallelize=True,
                   max_text_length=None,
                   min_text_length=None,
                   max_wave_length=80000,
                   shuffle=True,
                   random_shift_min=-4000,
                   random_shift_max= 4000,
                   random_stretch_min=0.7,
                   random_stretch_max= 1.3,
                   random_noise=0.75,
                   random_noise_factor_min=0.2,
                   random_noise_factor_max=0.5,
                   augment=False):
    return {
        'parallelize': parallelize,
        'shuffle': shuffle,
        'max_text_length': max_text_length,
        'min_text_length': min_text_length,
        'max_wave_length': max_wave_length,
        'random_shift_min': random_shift_min,
        'random_shift_max': random_shift_max,
        'random_stretch_min': random_stretch_min,
        'random_stretch_max': random_stretch_max,
        'random_noise': random_noise,
        'random_noise_factor_min': random_noise_factor_min,
        'random_noise_factor_max': random_noise_factor_max,
        'epochs': epochs,
        'batch_size': batch_size,
        'augment': augment
    }

In [ ]:
def experiment_params(data,
                      optimizer='Adam',
                      lr=1e-4,
                      alphabet=" 'abcdefghijklmnopqrstuvwxyz",
                      causal_convolutions=True,
                      stack_dilation_rates= [1, 3, 9, 27, 81],
                      stacks=2,
                      stack_kernel_size= 3,
                      stack_filters= 32,
                      sampling_rate=16000,
                      n_fft=160*4,
                      frame_step=160,
                      lower_edge_hertz=0,
                      upper_edge_hertz=8000,
                      num_mel_bins=160,
                      clip_gradients=None,
                      codename='regular',
                      **kwargs):
    params = {
        'optimizer': optimizer,
        'lr': lr,
        'data': data,
        'alphabet': alphabet,
        'causal_convolutions': causal_convolutions,
        'stack_dilation_rates': stack_dilation_rates,
        'stacks': stacks,
        'stack_kernel_size': stack_kernel_size,
        'stack_filters': stack_filters,
        'sampling_rate': sampling_rate,
        'n_fft': n_fft,
        'frame_step': frame_step,
        'lower_edge_hertz': lower_edge_hertz,
        'upper_edge_hertz': upper_edge_hertz,
        'num_mel_bins': num_mel_bins,
        'clip_gradients': clip_gradients,
        'codename': codename
    }
    
    #import pdb; pdb.set_trace()
    
    if kwargs is not None and 'data' in kwargs:
        params['data'] = { **params['data'], **kwargs['data'] }
        del kwargs['data']
        
    if kwargs is not None:
        params = { **params, **kwargs }
        
    return params

In [ ]:
import copy

def experiment(data_params=dataset_params(), **kwargs):
    params = experiment_params(
        data_params,
        **kwargs
    )
    
    print(params)

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir='stats/{}'.format(experiment_name(params)),
        params=params
    )
    
    #import pdb; pdb.set_trace()
    
    train_spec = tf.estimator.TrainSpec(
        input_fn=input_fn(
            train_data,
            params['data']
        )
    )
    
    features = {
        "audio": tf.placeholder(dtype=tf.float32, shape=[None]),
        "length": tf.placeholder(dtype=tf.int32, shape=[])
    }
    
    serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
        features
    )
    
    best_exporter = tf.estimator.BestExporter(
      name="best_exporter",
      serving_input_receiver_fn=serving_input_receiver_fn,
      exports_to_keep=5
    )
    
    eval_params = copy.deepcopy(params['data'])
    eval_params['augment'] = False
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn=input_fn(
            eval_data,
            eval_params
        ),
        throttle_secs=60*30,
        exporters=best_exporter
    )
    
    tf.estimator.train_and_evaluate(
        estimator,
        train_spec,
        eval_spec
    )

In [ ]:
def test(data_params=dataset_params(), **kwargs):
    params = experiment_params(
        data_params,
        **kwargs
    )
    
    print(params)

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir='stats/{}'.format(experiment_name(params)),
        params=params
    )
    
    eval_params = copy.deepcopy(params['data'])
    eval_params['augment'] = False
    eval_params['epochs'] = 1
    eval_params['shuffle'] = False

    estimator.evaluate(
        input_fn=input_fn(
            test_data,
            eval_params
        )
    )

In [ ]:
def predict_test(**kwargs):
    params = experiment_params(
        dataset_params(
            augment=False,
            shuffle=False,
            batch_size=1,
            epochs=1,
            parallelize=False
        ),
        **kwargs
    )
    
    print(len(test_data))

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir='stats/{}'.format(experiment_name(params)),
        params=params
    )

    return list(
        estimator.predict(
            input_fn=input_fn(
                test_data,
                params['data']
            )
        )
    )

In [ ]:
def predict(filepath, **kwargs):
    params = experiment_params(
        dataset_params(
            augment=False,
            shuffle=False,
            batch_size=1,
            epochs=1,
            parallelize=False
        ),
        **kwargs
    )
    
    dataset = pd.DataFrame(columns=['filename', 'text'])
    dataset['filename'] = [filepath]
    dataset['text'] = ['']
    
    print(len(dataset))

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir='stats/{}'.format(experiment_name(params)),
        params=params
    )

    return list(
        estimator.predict(
            input_fn=input_fn(
                dataset,
                params['data']
            )
        )
    )

In [ ]:
def eager_experiment(data_params=dataset_params(), **kwargs):
    params = experiment_params(
        data_params,
        **kwargs
    )
    
    print(params)

    dataset=input_fn(
        train_data,
        params['data']
    )()
    
    features, labels = dataset.make_one_shot_iterator().get_next()
    
    model_fn(features, labels, tf.estimator.ModeKeys.TRAIN, params)

In [ ]:
!pip install hypothesis

In [ ]:
import unittest
from hypothesis import given, settings, note, assume, reproduce_failure
import hypothesis.strategies as st
import hypothesis.extra.numpy as npst

# assuming test path will look like: 1/file.wav
def dummy_load_wave(example):
    row, params = example
    path = row.filename
    
    return np.ones((SAMPLING_RATE)) * float(path.split('/')[0]), row

class TestNotebook(unittest.TestCase):
    
    @given(
        st.sampled_from([22000, 16000, 8000]),
        st.sampled_from([1024, 512]),
        st.sampled_from([1024, 512]),
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        )
    )
    @settings(max_examples=5)
    def test_log_mel_conversion_works(self, sampling_rate, n_fft, frame_step, audio):
        lower_edge_hertz=0.0
        upper_edge_hertz=sampling_rate / 2.0
        num_mel_bins=64
        
        def librosa_melspectrogram(audio_item):
            spectrogram = np.abs(
                librosa.core.stft(
                    audio_item,
                    n_fft=n_fft,
                    hop_length=frame_step,
                    center=False
                )
            )**2
            
            return np.log(
                librosa.feature.melspectrogram(
                    S=spectrogram,
                    sr=sampling_rate,
                    n_mels=num_mel_bins,
                    fmin=lower_edge_hertz,
                    fmax=upper_edge_hertz,
                ) + 1e-6
            )
            
        audio_ph = tf.placeholder(tf.float32, (4, 16000))
        
        librosa_log_mels = np.transpose(
            np.stack([
                librosa_melspectrogram(audio_item)
                for audio_item in audio
            ]),
            (0, 2, 1)
        )
        
        log_mel_op = tf.check_numerics(
            LogMelSpectrogram(
                sampling_rate=sampling_rate,
                n_fft=n_fft,
                frame_step=frame_step,
                lower_edge_hertz=lower_edge_hertz,
                upper_edge_hertz=upper_edge_hertz,
                num_mel_bins=num_mel_bins
            )(audio_ph),
            message="log mels"
        )
        
        with tf.Session() as session:
            session.run(tf.global_variables_initializer())
            
            log_mels = session.run(
                log_mel_op,
                {
                   audio_ph: audio
                }
            )
            
            np.testing.assert_allclose(
                log_mels,
                librosa_log_mels,
                rtol=1e-1,
                atol=0
            )
    
    @given(
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        ),
        st.sampled_from([64, 32]),
        st.sampled_from([7, 3]),
        st.sampled_from([1, 4]),
    )
    @settings(max_examples=10)
    def test_residual_block_works(self, audio_waves, filters, size, dilation_rate):
        with tf.Graph().as_default() as g:
            audio_ph = tf.placeholder(tf.float32, (4, None))

            log_mel_op = LogMelSpectrogram(
                sampling_rate=16000,
                n_fft=512,
                frame_step=256,
                lower_edge_hertz=0,
                upper_edge_hertz=8000,
                num_mel_bins=10
            )(audio_ph)

            expanded_op = tf.layers.Dense(filters)(log_mel_op)

            _, block_op = ResidualBlock(
                filters=filters,
                kernel_size=size,
                causal=True,
                dilation_rate=dilation_rate
            )(expanded_op, training=True)

            # really dumb loss function just for the sake
            # of testing:
            loss_op = tf.reduce_sum(block_op)

            variables = tf.trainable_variables()
            self.assertTrue(any(["batch_normalization" in var.name for var in variables]))

            grads_op = tf.gradients(
                loss_op,
                variables
            )
        
            for grad, var in zip(grads_op, variables):
                if grad is None:
                    note(var)

                self.assertTrue(grad is not None)

            with tf.Session(graph=g) as session:
                session.run(tf.global_variables_initializer())

                result, expanded, grads, _ = session.run(
                    [block_op, expanded_op, grads_op, loss_op],
                    {
                        audio_ph: audio_waves
                    }
                )

                self.assertFalse(np.array_equal(result, expanded))
                self.assertEqual(result.shape, expanded.shape)
                self.assertEqual(len(grads), len(variables))
                self.assertFalse(any([np.isnan(grad).any() for grad in grads]))
    
    @given(
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        ),
        st.sampled_from([64, 32]),
        st.sampled_from([7, 3])
    )
    @settings(max_examples=10)
    def test_residual_stack_works(self, audio_waves, filters, size):
        dilation_rates = [1,2,4]
        
        with tf.Graph().as_default() as g:
            audio_ph = tf.placeholder(tf.float32, (4, None))

            log_mel_op = LogMelSpectrogram(
                sampling_rate=16000,
                n_fft=512,
                frame_step=256,
                lower_edge_hertz=0,
                upper_edge_hertz=8000,
                num_mel_bins=10
            )(audio_ph)

            expanded_op = tf.layers.Dense(filters)(log_mel_op)

            stack_op = ResidualStack(
                filters=filters,
                kernel_size=size,
                causal=True,
                dilation_rates=dilation_rates
            )(expanded_op, training=True)

            # really dumb loss function just for the sake
            # of testing:
            loss_op = tf.reduce_sum(stack_op)

            variables = tf.trainable_variables()
            self.assertTrue(any(["batch_normalization" in var.name for var in variables]))

            grads_op = tf.gradients(
                loss_op,
                variables
            )
        
            for grad, var in zip(grads_op, variables):
                if grad is None:
                    note(var)

                self.assertTrue(grad is not None)

            with tf.Session(graph=g) as session:
                session.run(tf.global_variables_initializer())

                result, expanded, grads, _ = session.run(
                    [stack_op, expanded_op, grads_op, loss_op],
                    {
                        audio_ph: audio_waves
                    }
                )

                self.assertFalse(np.array_equal(result, expanded))
                self.assertEqual(result.shape, expanded.shape)
                self.assertEqual(len(grads), len(variables))
                self.assertFalse(any([np.isnan(grad).any() for grad in grads]))
    
    def test_causal_conv1d_works(self):
        conv_size2_dilation_1 = AtrousConv1D(
            filters=1,
            kernel_size=2,
            dilation_rate=1,
            kernel_initializer=tf.ones_initializer(),
            use_bias=False
        )
        
        conv_size3_dilation_1 = AtrousConv1D(
            filters=1,
            kernel_size=3,
            dilation_rate=1,
            kernel_initializer=tf.ones_initializer(),
            use_bias=False
        )
        
        conv_size2_dilation_2 = AtrousConv1D(
            filters=1,
            kernel_size=2,
            dilation_rate=2,
            kernel_initializer=tf.ones_initializer(),
            use_bias=False
        )
        
        conv_size2_dilation_3 = AtrousConv1D(
            filters=1,
            kernel_size=2,
            dilation_rate=3,
            kernel_initializer=tf.ones_initializer(),
            use_bias=False
        )
        
        data = np.array(list(range(1, 31)))
        data_ph = tf.placeholder(tf.float32, (1, 30, 1))
        
        size2_dilation_1_1 = conv_size2_dilation_1(data_ph)
        size2_dilation_1_2 = conv_size2_dilation_1(size2_dilation_1_1)
        
        size3_dilation_1_1 = conv_size3_dilation_1(data_ph)
        size3_dilation_1_2 = conv_size3_dilation_1(size3_dilation_1_1)
        
        size2_dilation_2_1 = conv_size2_dilation_2(data_ph)
        size2_dilation_2_2 = conv_size2_dilation_2(size2_dilation_2_1)
        
        size2_dilation_3_1 = conv_size2_dilation_3(data_ph)
        size2_dilation_3_2 = conv_size2_dilation_3(size2_dilation_3_1)
        
        with tf.Session() as session:        
            session.run(tf.global_variables_initializer())
            
            outputs = session.run(
                [
                    size2_dilation_1_1,
                    size2_dilation_1_2,
                    size3_dilation_1_1,
                    size3_dilation_1_2,
                    size2_dilation_2_1,
                    size2_dilation_2_2,
                    size2_dilation_3_1,
                    size2_dilation_3_2
                ],
                {
                    data_ph: np.reshape(data, (1, 30, 1))
                }
            )
            
            for ix, out in enumerate(outputs):
                out = np.squeeze(out)
                outputs[ix] = out
                
                self.assertEqual(out.shape[0], len(data))
            
            np.testing.assert_equal(
                outputs[0],
                np.array([1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[1],
                np.array([1, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[2],
                np.array([1, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[3],
                np.array([1, 4, 10, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144, 153, 162, 171, 180, 189, 198, 207, 216, 225, 234, 243, 252], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[4],
                np.array([1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[5],
                np.array([1, 2, 5, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[6],
                np.array([1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57], dtype=np.float32)
            )

            np.testing.assert_equal(
                outputs[7],
                np.array([1, 2, 3, 6, 9, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108], dtype=np.float32)
            )
    
    @given(
        npst.arrays(
            np.float32,
            (4, 30, len('abcdefghijk1234!@#$%^&*')),
            elements=st.floats(0, 1)
        )
    )
    @settings(max_examples=10)
    def test_decode_logits_doesnt_crash(self, logits):
        params = { 'alphabet': 'abcdefghijk1234!@#$%^&*' }
        
        lengths = np.array([15, 15, 15, 15], dtype=np.int32)
        
        logits_ph = tf.placeholder(
            tf.float32,
            shape=(4, 30, len(params['alphabet']))
        )
        
        lengths_ph = tf.placeholder(
            tf.int32,
            shape=(4)
        )
        
        decode_op, codes_op = decode_logits(
            logits_ph,
            lengths_ph,
            params
        )
        
        with tf.Session() as session:        
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer(name='init_all_tables'))
            
            codes, decoded = session.run(
                [codes_op, decode_op],
                {
                    logits_ph: logits,
                    lengths_ph: lengths
                }
            )
            
            results = np.ones(codes.dense_shape) * -1
            
            for ix, value in zip(codes.indices, codes.values):
                results[ix[0], ix[1]] = value

            for row in results:
                self.assertLessEqual(len(row[row != -1]), 15)
    
    @given(st.text(alphabet="!\"&',-.01234:;\abcdefghijklmnopqrstuvwxyz", max_size=10))
    @settings(max_examples=10)
    def test_encode_and_decode_work(self, text):
        assume(text != '')
        
        params = { 'alphabet': '!"&\',-.01234:;\abcdefghijklmnopqrstuvwxyz' }
        
        label_ph = tf.placeholder(tf.string, shape=(1), name='text')
        codes_op = encode_labels(label_ph, params)
        decode_op = decode_codes(codes_op, params)

        with tf.Session() as session:        
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer(name='init_all_tables'))
            
            codes, decoded = session.run(
                [codes_op, decode_op],
                {
                    label_ph: np.array([text])
                }
            )
            
            note(codes)
            note(decoded)
            
            self.assertEqual(text, ''.join(map(lambda s: s.decode('UTF-8'), decoded.values)))
            self.assertEqual(codes.values.dtype, np.int32)
            self.assertEqual(len(codes.values), len(text))
    
    @given(
        npst.arrays(
            np.float32,
            (st.integers(min_value=16000, max_value=16000*5)),
            elements=st.floats(-1, 1)
        ),
        st.sampled_from([22000, 16000, 8000]),
        st.sampled_from([1024, 512, 640]),
        st.sampled_from([1024, 512, 160]),
    )
    @settings(max_examples=10)
    def test_compute_lengths_works(self,
                                   audio_wave,
                                   sampling_rate,
                                   n_fft,
                                   frame_step
                                  ):
        assume(n_fft >= frame_step)
        
        original_wave_length = audio_wave.shape[0]
        
        audio_waves_ph = tf.placeholder(tf.float32, (None, None), name="audio_waves")
        original_lengths_ph = tf.placeholder(tf.int32, (None), name="original_lengths")
        
        lengths_op = compute_lengths(
            original_lengths_ph,
            {
                'frame_step': frame_step,
                'n_fft': n_fft
            }
        )
        
        self.assertEqual(lengths_op.dtype, tf.int32)
        
        log_mel_op = LogMelSpectrogram(
            sampling_rate=sampling_rate,
            n_fft=n_fft,
            frame_step=frame_step,
            lower_edge_hertz=0.0,
            upper_edge_hertz=8000.0,
            num_mel_bins=13
        )(audio_waves_ph)
        
        with tf.Session() as session:        
            session.run(tf.global_variables_initializer())
            
            lengths, log_mels = session.run(
                [lengths_op, log_mel_op],
                {
                    audio_waves_ph: np.array([audio_wave]),
                    original_lengths_ph: np.array([original_wave_length])
                }
            )
            
            note(original_wave_length)
            note(lengths)
            note(log_mels.shape)
            
            self.assertEqual(lengths[0], log_mels.shape[1])
    
    @given(
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        )
    )
    @settings(max_examples=10)
    def test_speech_net_works(self, audio_waves):
        with tf.Graph().as_default() as g:
            audio_ph = tf.placeholder(tf.float32, (4, None))

            logits_op = SpeechNet(
                experiment_params(
                    {},
                    stack_dilation_rates= [1, 2, 4],
                    stack_kernel_size= 3,
                    stack_filters= 32,
                    alphabet= 'abcd'
                )
            )(audio_ph)

            # really dumb loss function just for the sake
            # of testing:
            loss_op = tf.reduce_sum(logits_op)

            variables = tf.trainable_variables()
            self.assertTrue(any(["batch_normalization" in var.name for var in variables]))

            grads_op = tf.gradients(
                loss_op,
                variables
            )
        
            for grad, var in zip(grads_op, variables):
                if grad is None:
                    note(var)

                self.assertTrue(grad is not None)

            with tf.Session(graph=g) as session:
                session.run(tf.global_variables_initializer())

                result, grads, _ = session.run(
                    [logits_op, grads_op, loss_op],
                    {
                        audio_ph: audio_waves
                    }
                )

                self.assertEqual(result.shape[2], 5)
                self.assertEqual(len(grads), len(variables))
                self.assertFalse(any([np.isnan(grad).any() for grad in grads]))
                
    @given(
        npst.arrays(
            np.float32,
            (4, 16000),
            elements=st.floats(-1, 1)
        )
    )
    @settings(max_examples=10)
    def test_batch_normalization_when_not_training_doesnt_have_gradients(self, audio_waves):
        with tf.Graph().as_default() as g:
            audio_ph = tf.placeholder(tf.float32, (4, None))

            logits_op = SpeechNet(
                experiment_params(
                    {},
                    stack_dilation_rates= [1, 2, 4],
                    stack_kernel_size= 3,
                    stack_filters= 32,
                    alphabet= 'abcd'
                )
            )(audio_ph, training=False)

            # really dumb loss function just for the sake
            # of testing:
            loss_op = tf.reduce_sum(logits_op)

            variables = tf.trainable_variables()

            grads_op = tf.gradients(
                loss_op,
                variables
            )
        
            for grad, var in zip(grads_op, variables):
                if grad is None:
                    note(var)

                self.assertTrue(grad is not None)

            with tf.Session(graph=g) as session:
                session.run(tf.global_variables_initializer())

                result, grads, _ = session.run(
                    [logits_op, grads_op, loss_op],
                    {
                        audio_ph: audio_waves
                    }
                )
                
                no_batch_norms = list(
                    filter(
                        lambda var: 'batch_normaslization' not in var.name,
                        variables
                    )
                )

                self.assertEqual(len(grads), len(no_batch_norms))
                self.assertFalse(any([np.isnan(grad).any() for grad in grads]))
    
    def test_dataset_returns_data_in_order(self):
        
        params = experiment_params(
            dataset_params(
                batch_size=2,
                epochs=1,
                augment=False
            )
        )
        
        data = pd.DataFrame(
            data={
                'text': [ str(i) for i in range(10)],
                'filename':  [ '{}/wav'.format(i) for i in range(10)]
            }
        )
        
        dataset = input_fn(data, params['data'], dummy_load_wave)()
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()
        
        with tf.Session() as session:
            try:
                while True:
                    audio, label = session.run(next_element)
                    audio, length = audio
                    
                    for _audio, _label in zip(list(audio), list(label)):
                        self.assertEqual(_audio[0], float(_label))
                    
                    for _length in length:
                        self.assertEqual(_length, SAMPLING_RATE)
            except tf.errors.OutOfRangeError:
                pass

In [ ]:
if __name__ == '__main__' and RUN_TESTS:
    import doctest
    
    doctest.testmod()
    unittest.main(
        argv=['first-arg-is-ignored', 'TestNotebook.test_encode_and_decode_work'],
        failfast=True,
        exit=False
    )

In [ ]:
#import glob
#
#def get_len(filename):
#    return len(hkl.load(filename))
#
#lengths = [
#    get_len(filename)
#    for filename in glob.glob('data/cv_corpus_v1/cv-valid-train/*.wave.hkl')
#]

In [ ]:
#np.min(lengths)

In [ ]:
#np.max(lengths)

In [ ]:
#import seaborn as sb
#
#sb.distplot(lengths, kde = False)

### Experiments

In [ ]:
experiment(
    dataset_params(
        batch_size=18,
        epochs=10,
        max_wave_length=320000,
        augment=True,
        random_noise=0.75,
        random_noise_factor_min=0.1,
        random_noise_factor_max=0.15,
        random_stretch_min=0.8,
        random_stretch_max=1.2
    ),
    codename='deep_max_20_seconds',
    alphabet=' !"&\',-.01234:;\\abcdefghijklmnopqrstuvwxyz', # !"&',-.01234:;\abcdefghijklmnopqrstuvwxyz
    causal_convolutions=False,
    stack_dilation_rates=[1, 3, 9, 27],
    stacks=6,
    stack_kernel_size=7,
    stack_filters=3*128,
    n_fft=160*8,
    frame_step=160*4,
    num_mel_bins=160,
    optimizer='Momentum',
    lr=0.00001,
    clip_gradients=20.0
)

{'optimizer': 'Momentum', 'lr': 1e-05, 'data': {'parallelize': True, 'shuffle': True, 'max_text_length': None, 'min_text_length': None, 'max_wave_length': 320000, 'random_shift_min': -4000, 'random_shift_max': 4000, 'random_stretch_min': 0.8, 'random_stretch_max': 1.2, 'random_noise': 0.75, 'random_noise_factor_min': 0.1, 'random_noise_factor_max': 0.15, 'epochs': 10, 'batch_size': 18, 'augment': True}, 'alphabet': ' !"&\',-.01234:;\\abcdefghijklmnopqrstuvwxyz', 'causal_convolutions': False, 'stack_dilation_rates': [1, 3, 9, 27], 'stacks': 6, 'stack_kernel_size': 7, 'stack_filters': 384, 'sampling_rate': 16000, 'n_fft': 1280, 'frame_step': 640, 'lower_edge_hertz': 0, 'upper_edge_hertz': 8000, 'num_mel_bins': 160, 'clip_gradients': 20.0, 'codename': 'deep_max_20_seconds'}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'stats/causal_convolutions_False/codename_deep_max_20_seconds/frame_step_640/lower_edge_hertz_0/n_fft_1280/num_mel_bins_160/optimizer_M

Process ForkPoolWorker-12:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
Process ForkPoolWorker-11:
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.6/

KeyboardInterrupt: ignored

> /tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py(1443)_call_tf_sessionrun()
   1441     return tf_session.TF_SessionRun_wrapper(self._session, options, feed_dict,
   1442                                             fetch_list, target_list,
-> 1443                                             run_metadata)
   1444 
   1445   def _call_tf_sessionprun(self, handle, feed_dict, fetch_list):

--KeyboardInterrupt--


Process ForkPoolWorker-36:
Process ForkPoolWorker-32:
Process ForkPoolWorker-31:
Process ForkPoolWorker-26:
Process ForkPoolWorker-33:
Process ForkPoolWorker-25:
Process ForkPoolWorker-27:
Process ForkPoolWorker-35:
Process ForkPoolWorker-30:
Process ForkPoolWorker-34:
Process ForkPoolWorker-29:
Process ForkPoolWorker-28:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._k

--KeyboardInterrupt--


Process ForkPoolWorker-43:
Process ForkPoolWorker-44:
Process ForkPoolWorker-37:
Process ForkPoolWorker-40:
Process ForkPoolWorker-38:
Process ForkPoolWorker-39:
Process ForkPoolWorker-41:
Process ForkPoolWorker-48:
Process ForkPoolWorker-45:
Process ForkPoolWorker-47:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bo

In [ ]:
test(
    dataset_params(
        batch_size=18,
        epochs=10,
        max_wave_length=320000,
        augment=True,
        random_noise=0.75,
        random_noise_factor_min=0.1,
        random_noise_factor_max=0.15,
        random_stretch_min=0.8,
        random_stretch_max=1.2
    ),
    codename='deep_max_20_seconds',
    alphabet=' !"&\',-.01234:;\\abcdefghijklmnopqrstuvwxyz', # !"&',-.01234:;\abcdefghijklmnopqrstuvwxyz
    causal_convolutions=False,
    stack_dilation_rates=[1, 3, 9, 27],
    stacks=6,
    stack_kernel_size=7,
    stack_filters=3*128,
    n_fft=160*8,
    frame_step=160*4,
    num_mel_bins=160,
    optimizer='Momentum',
    lr=0.00001,
    clip_gradients=20.0
)

{'upper_edge_hertz': 8000, 'clip_gradients': 20.0, 'frame_step': 640, 'data': {'random_noise_factor_max': 0.15, 'max_text_length': None, 'epochs': 10, 'random_shift_min': -4000, 'random_noise_factor_min': 0.1, 'batch_size': 18, 'max_wave_length': 320000, 'augment': True, 'random_stretch_min': 0.8, 'min_text_length': None, 'random_noise': 0.75, 'shuffle': True, 'random_stretch_max': 1.2, 'parallelize': True, 'random_shift_max': 4000}, 'sampling_rate': 16000, 'alphabet': ' !"&\',-.01234:;\\abcdefghijklmnopqrstuvwxyz', 'codename': 'deep_max_20_seconds', 'stack_filters': 384, 'lower_edge_hertz': 0, 'stack_dilation_rates': [1, 3, 9, 27], 'n_fft': 1280, 'optimizer': 'Momentum', 'num_mel_bins': 160, 'stack_kernel_size': 7, 'causal_convolutions': False, 'stacks': 6, 'lr': 1e-05}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_protocol': None, '_save_summary_steps': 100, '_experimental_distribute': None, '_model_dir': 'stats/causal_convolutions_Fals

In [ ]:
test_results = predict_test(
    codename='deep_max_20_seconds',
    alphabet=' !"&\',-.01234:;\\abcdefghijklmnopqrstuvwxyz', # !"&',-.01234:;\abcdefghijklmnopqrstuvwxyz
    causal_convolutions=False,
    stack_dilation_rates=[1, 3, 9, 27],
    stacks=6,
    stack_kernel_size=7,
    stack_filters=3*128,
    n_fft=160*8,
    frame_step=160*4,
    num_mel_bins=160,
    optimizer='Momentum',
    lr=0.00001,
    clip_gradients=20.0
)

3995
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_protocol': None, '_save_summary_steps': 100, '_experimental_distribute': None, '_model_dir': 'stats/causal_convolutions_False/codename_deep_max_20_seconds/frame_step_640/lower_edge_hertz_0/n_fft_1280/num_mel_bins_160/optimizer_Momentum/sampling_rate_16000/stack_dilation_rates_1_3_9_27/stack_filters_384/stack_kernel_size_7/stacks_6/upper_edge_hertz_8000', '_task_id': 0, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_master': '', '_global_id_in_cluster': 0, '_is_chief': True, '_tf_random_seed': None, '_evaluation_master': '', '_service': None, '_num_worker_replicas': 1, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.C

In [ ]:
[ b''.join(t['text']) for t in test_results ]

[b'without the dotaset the artice suistles',
 b"i've got to go to him",
 b'and you know it',
 b'down below in the darknes were hundrededs of people sleping in peace',
 b'strange images pased through my mind',
 b'the shep had taught him that',
 b'it was glaringly hot not a clou in hesky nor a breath of wind',
 b'your son went to serve at a distant place and became a cinturion',
 b'they made a boy continue tiging but he found nothing',
 b'the shoreas in da',
 b'fol the instructions here',
 b"the're caling to u not to give up and to kep on fighting",
 b'the shop was closed on monis',
 b'even coming down on the train together she wrote me',
 b"i'm going away he said",
 b"he wasn't asking for help",
 b'some of the grynsh was faling of the circular edge',
 b"i'd like to think",
 b'the alchemist robably already knew al that',
 b"you 'l take fiftly and like et",
 b'it was droping of in flakes and raining down on the sand',
 b"what's your name he asked",
 b"it's because you were not born",
 b'w